# Intro
Welcome to the [iWildcam 2021 - FGVC8](https://www.kaggle.com/c/iwildcam2021-fgvc8) compedition.
![](https://storage.googleapis.com/kaggle-competitions/kaggle/24911/logos/header.png)

Informations about the data structure you will get on the [github](https://github.com/visipedia/iwildcam_comp/blob/master/readme.md) repository.

<span style="color: royalblue;">Please vote the notebook up if it helps you. Feel free to leave a comment above the notebook. Thank you. </span>

# Libraries

In [ ]:
import os
import pandas as pd
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
from collections import Counter

# Path

In [ ]:
path = '/kaggle/input/iwildcam2021-fgvc8/'
os.listdir(path)

In [ ]:
os.listdir(path+'metadata')

# Load Data

In [ ]:
with open(path+'metadata/'+'iwildcam2021_train_annotations.json') as f:
    train_data = json.load(f)
with open(path+'metadata/'+'iwildcam2021_test_information.json') as f:
    test_data = json.load(f)
with open(path+'metadata/'+'iwildcam2021_megadetector_results.json') as f:
    results_data = json.load(f)

Write image metadata into dataframe:

In [ ]:
train_image_data = pd.json_normalize(train_data['images'])
train_image_data.index=train_image_data['id']

# Functions
We define some helper functions.

In [ ]:
def plot_examples():
    fig, axs = plt.subplots(4, 4, figsize=(20, 20))
    fig.subplots_adjust(hspace = .1, wspace=.1)
    
    axs = axs.ravel()
    for i in range(16):
        img = cv2.imread(path+'train/'+results_data['images'][i]['id']+'.jpg')
        axs[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if len(results_data['images'][i]['detections']) > 0:
            
            width = train_image_data.loc[results_data['images'][i]['id'], 'width']
            height = train_image_data.loc[results_data['images'][i]['id'], 'height']
            for dets in range(len(results_data['images'][i]['detections'])):
                bbox = results_data['images'][i]['detections'][dets]['bbox']            
                p = matplotlib.patches.Rectangle((width*bbox[0], height*bbox[1]),
                                              width*bbox[2],
                                              height*bbox[3],
                                              ec='r', fc='none', lw=2.)
                axs[i].add_patch(p)
                axs[i].set_title('category '+results_data['images'][i]['detections'][dets]['category'])
        else:
            axs[i].set_title('no detections')
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
    plt.show()

# Overview

In [ ]:
print('Number train images:', len(os.listdir(path+'train/')))
print('Number test images:', len(os.listdir(path+'test/')))

# EDA

## Plot Image And Bounding Boxes

In [ ]:
plot_examples()

## Distribution Of Categories

In [ ]:
train_cate = []
for row in range(len(results_data['images'])):
    if len(results_data['images'][row]['detections']) > 0:
        for dets in range(len(results_data['images'][row]['detections'])):
            train_cate.append(results_data['images'][row]['detections'][dets]['category'])
    else:
        train_cate.append('no_detection')
label_dist = Counter(train_cate)
label_dist